In [1]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from tqdm.auto import tqdm
import torch.nn.functional as F
import Models.Network as models
from torchsummary import summary

/data4/home/hrishikeshj/DLNLP/envForDLNLP/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loading Dataset

dataroot = r"/data4/home/hrishikeshj/DLNLP/ADRLA2/Dataset/train"
image_size = 128
batch_size = 64

dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=8, drop_last=True)

In [3]:
dim_of_emb = 128
device_to_use = 1

In [4]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class projection_MLP(nn.Module):
    def __init__(self):
        super(projection_MLP, self).__init__()

        n_channels = 512

        self.projection_head = nn.Sequential()

        self.projection_head.add_module('W1', nn.Linear(
            n_channels, n_channels))
        self.projection_head.add_module('ReLU', nn.ReLU())
        self.projection_head.add_module('W2', nn.Linear(
            n_channels, 128))

    def forward(self, x):
        return self.projection_head(x)

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=128, zero_init_residual=False,
                 groups=1, width_per_group=64):
        super(ResNet, self).__init__()
        
        self._norm_layer = nn.BatchNorm2d

        self.register_buffer

        self.inplanes = 64
        self.dilation = 1
        
        self.groups = groups
        self.base_width = width_per_group

        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                                   bias=False)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

            

        self.bn1 = self._norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(512 * block.expansion,num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                # if isinstance(m, Bottleneck):
                #     nn.init.constant_(m.bn3.weight, 0)
                if isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = torch.flatten(x, 1)

        x = self.fc(x)

        return x

In [5]:
queue_size = 128*batch_size
momentum = 0.999
temperature = 0.07

learning_rate = 0.03
sgd_weight_decay = 0.0001
sgd_momentum = 0.9
num_epochs = 100

In [6]:
class MoCo_Model(nn.Module):
    def __init__(self):
        
        super(MoCo_Model, self).__init__()

        self.queue_size = queue_size
        self.momentum = momentum
        self.temperature = temperature

        assert self.queue_size % batch_size == 0  # for simplicity

        # Load model
        self.query_encoder = ResNet(BasicBlock,[2,2,2,2],num_classes=dim_of_emb)
        self.key_encoder = ResNet(BasicBlock,[2,2,2,2],num_classes=dim_of_emb)

        self.query_encoder.fc = projection_MLP()
        self.key_encoder.fc = projection_MLP()


        # Initialize the key encoder to have the same values as query encoder
        # Do not update the key encoder via gradient
        for query_param, key_param in zip(self.query_encoder.parameters(), self.key_encoder.parameters()):
            key_param.data.copy_(query_param.data)
            key_param.requires_grad = False

        # Create the queue to store negative samples
        self.register_buffer("queue", torch.randn(self.queue_size, dim_of_emb))

        # Create pointer to store current position in the queue when enqueue and dequeue
        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

    @torch.no_grad()
    def momentum_update(self):
        
        # For each of the parameters in each encoder
        for p_q, p_k in zip(self.query_encoder.parameters(), self.key_encoder.parameters()):
            p_k.data = p_k.data * self.momentum + p_q.detach().data * (1. - self.momentum)

    @torch.no_grad()
    def shuffled_idx(self, batch_size):
        '''
        Generation of the shuffled indexes for the implementation of ShuffleBN.

        https://github.com/HobbitLong/CMC.

        args:
            batch_size (Tensor.int()):  Number of samples in a batch

        returns:
            shuffled_idxs (Tensor.long()): A random permutation index order for the shuffling of the current minibatch

            reverse_idxs (Tensor.long()): A reverse of the random permutation index order for the shuffling of the
                                            current minibatch to get back original sample order

        '''

        # Generate shuffled indexes
        shuffled_idxs = torch.randperm(batch_size).long().cuda()

        reverse_idxs = torch.zeros(batch_size).long().cuda()

        value = torch.arange(batch_size).long().cuda()

        reverse_idxs.index_copy_(0, shuffled_idxs, value)

        return shuffled_idxs, reverse_idxs

    @torch.no_grad()
    def update_queue(self, feat_k):

        curr_BS = feat_k.size(0)

        ptr = int(self.queue_ptr)

        # replace the keys at ptr (dequeue and enqueue)
        self.queue[ptr:ptr + curr_BS, :] = feat_k

        # move pointer along to end of current batch
        ptr = (ptr + curr_BS) % self.queue_size

        # Store queue pointer as register_buffer
        self.queue_ptr[0] = ptr

    def InfoNCE_logits(self, f_q, f_k):

        f_k = f_k.detach()

        # Get queue from register_buffer
        f_mem = self.queue.clone().detach()

        # Normalize the feature representations
        f_q = nn.functional.normalize(f_q, dim=1)
        f_k = nn.functional.normalize(f_k, dim=1)
        f_mem = nn.functional.normalize(f_mem, dim=1)

        # Compute sim between positive views
        pos = torch.bmm(f_q.view(f_q.size(0), 1, -1),
                        f_k.view(f_k.size(0), -1, 1)).squeeze(-1)

        # Compute sim between postive and all negatives in the memory
        neg = torch.mm(f_q, f_mem.transpose(1, 0))

        logits = torch.cat((pos, neg), dim=1)

        logits /= self.temperature

        # Create labels, first logit is postive, all others are negative
        labels = torch.zeros(logits.shape[0], dtype=torch.long).cuda()

        return logits, labels

    def forward(self, x_q, x_k):

        batch_size = x_q.size(0)

        # Feature representations of the query view from the query encoder
        feat_q = self.query_encoder(x_q)

        # TODO: shuffle ids with distributed data parallel
        # Get shuffled and reversed indexes for the current minibatch
        shuffled_idxs, reverse_idxs = self.shuffled_idx(batch_size)

        with torch.no_grad():
            # Update the key encoder
            self.momentum_update()

            # Shuffle minibatch
            x_k = x_k[shuffled_idxs]

            # Feature representations of the shuffled key view from the key encoder
            feat_k = self.key_encoder(x_k)

            # reverse the shuffled samples to original position
            feat_k = feat_k[reverse_idxs]

        # Compute the logits for the InfoNCE contrastive loss.
        logit, label = self.InfoNCE_logits(feat_q, feat_k)

        # Update the queue/memory with the current key_encoder minibatch.
        self.update_queue(feat_k)

        return logit, label

In [7]:
mocoModel = MoCo_Model()

In [8]:
torch.cuda.set_device(device_to_use)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [9]:
mocoModel = mocoModel.to(device=device)

In [10]:
optimiser = torch.optim.SGD(mocoModel.parameters(), lr=learning_rate, weight_decay=sgd_weight_decay, momentum=sgd_momentum)

In [11]:
lr_decay = torch.optim.lr_scheduler.CosineAnnealingLR(optimiser, num_epochs)

In [12]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [13]:
augmentation = [
            transforms.RandomGrayscale(p=0.2),
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
            transforms.RandomHorizontalFlip(),
            transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]

augTransform = transforms.Compose(augmentation)

In [14]:
#Pretrain
number_of_steps = len(dataloader)*num_epochs
progress_bar = tqdm(range(number_of_steps))

mocoModel.train()
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(dataloader):
        optimiser.zero_grad()
        #images : [bs,3,128,128]
        images = images.to(device)
        #Get two augmentations:
        query_images = augTransform(images)
        key_images = augTransform(images)

        all_logits, labels = mocoModel(query_images,key_images)
        #all_logits : [(queue_size + 1),emb_dim]
        #labels : [(queue_size + 1)], Tensor of all zeros

        loss = criterion(all_logits,labels)

        loss.backward()

        optimiser.step()

        progress_bar.update()

        if i%500 == 0:
            print("Pretrain_loss : ",loss.item())

    lr_decay.step()


  0%|          | 3/22800 [00:33<54:33:09,  8.61s/it] 

Pretrain_loss :  0.02857186272740364


  1%|          | 230/22800 [02:23<2:03:29,  3.05it/s]

Pretrain_loss :  5.432933807373047


  2%|▏         | 456/22800 [02:42<29:20, 12.69it/s]  

...

Pretrain_loss :  0.27680158615112305


 99%|█████████▉| 22577/22800 [48:14<01:11,  3.13it/s]

Pretrain_loss :  0.211419016122818


100%|█████████▉| 22799/22800 [48:56<00:00,  5.73it/s]

100%|██████████| 22800/22800 [49:10<00:00,  5.73it/s]

In [17]:
ModelPath = "/data4/home/hrishikeshj/DLNLP/ADRLA3/trainedModels/TR1/mocoModel.pt"

In [18]:
torch.save(mocoModel.state_dict(),ModelPath)

In [15]:
augTransform = transforms.Compose(augmentation)


In [17]:
for onesample in dataloader:
    break

In [18]:
imageBatch = onesample[0].to(device)

In [19]:
x_q = augTransform(imageBatch)

/data4/home/hrishikeshj/DLNLP/envForDLNLP/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [20]:
x_k = augTransform(imageBatch)

In [21]:
op = mocoModel(x_q,x_k)

In [23]:
op[0].shape

torch.Size([64, 8193])

In [25]:
loss = criterion(op[0],op[1])

In [29]:
op[0]

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       grad_fn=<DivBackward0>)